Pytrends implementation - a warning: do not build too many payloads as Google will block your ip address and think you are a bot that's trying to spam its systems (error 429). 

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import os
import time
from dateutil.relativedelta import *
from dateutil.rrule import *
from datetime import date

Timeframe setup. A timeframe over 9 months will condense the data into weekly markers - not very good for our use. Timeframes are 9 months so they are still daily markers.

In [ ]:
use_date = date(2016,1,1)
today = date.today()
#beginning period of 9 months
begin = list(rrule(freq=MONTHLY, interval=8,count=10, dtstart=use_date))
for i in range(len(begin)):
    begin[i] = begin[i].date()

#calculate the last day of last month
use_date = use_date+relativedelta(months=-1)
use_date = use_date+relativedelta(day=31)

#end period of 9 months
end = list(rrule(freq=MONTHLY, interval=8,count=11, dtstart=use_date, bymonthday=(-1,)))
for i in range(len(end)):
    end[i] = end[i].date()

#adds time periods to a list
#if the end date is later than today's date, today's date will be used
dates = []
for i in range(len(begin)):
    if today >= begin[i] and today >= end[i+1]:
        full = (str(begin[i]), str(end[i+1]))
        dates.append(full)
    elif today >= begin[i] and today <= end[i+1]:
        recent = (str(begin[i]), str(today))
        dates.append(recent)
for i in dates:
    print(i)

Data processing. Creates a 'data.csv' file to store the data for each 9 month period while mantaining its daily marker. The keywords come from 'keywords.txt'.

In [ ]:
filename = 'keywords.txt'
with open(filename) as f:
    content = f.readlines()
for i in range(len(content)):
    content[i] = content[i].strip()

In [ ]:
print(content)

Constructing the payload with the keywords lists. After constructing the payload, it will be written to a csv file with the name of 'data_.csv' where the _ is the keyword. It will iterate through the keywords, then iterate through each 9 month time period so it maintains its daily markers.

In [ ]:
pytrend = TrendReq()
csvname = 'data'
allcsv = []
for i in content: 
    kw = []
    kw.append(i)
    csvfile = csvname + ' ' + i + '.csv'
    allcsv.append(csvfile)
    for j in dates:
        timef = ''
        timef = j[0] + ' ' +  j[1]           
        time.sleep(5) #if 0, or removed google will think you are a bot so make sure it's greater than 1
        pytrend.build_payload(kw,timeframe=timef)
        df = pytrend.interest_over_time().drop(['isPartial'],axis=1)
        try:
            if os.stat(csvfile).st_size > 0:
                df.to_csv(csvfile, mode='a', header=False)
            else:
                #file should never be empty
                print("empty file")
        except OSError:
            df.to_csv(csvfile,index=True)

In [ ]:
allcsvadj = []
for i in content:
    csvfile = csvname + ' ' + i + ' adjusted'+'.csv'
    colname = i + ' adjusted'
    allcsvadj.append(csvfile)
    print([i])
    for j in dates:
        overlap_start = None
        start = datetime.strptime(j[0], '%Y-%m-%d')
        initial_end = end = datetime.strptime(j[1], '%Y-%m-%d')
        temp_days = end - start        
        delta = timedelta(days=temp_days.days)
        overlap = timedelta(days=100) # number of days to overlap, might need to increase/perform some fine tuning
        iterate = end - delta
        df = pd.DataFrame()
        ol = pd.DataFrame()
        while end > start:
            timef = iterate.strftime('%Y-%m-%d')+' '+end.strftime('%Y-%m-%d')
            pytrend.build_payload([i],timeframe=timef)
            temp = pytrend.interest_over_time().drop(['isPartial'],axis=1)
            temp.columns.values[0] = timef
            ol_temp =temp.copy()
            ol_temp.iloc[:,:] = None
            if overlap_start is not None: # after first iteration
                y1 = temp.loc[overlap_start:end].iloc[:,0].values.max()
                y2 = df.loc[overlap_start:end].iloc[:,-1].values.max()
                coef = y2/y1 # coefficent to multiply by for scaling
                temp = temp * coef
                ol_temp.loc[overlap_start:end, :] = 1 
            df = pd.concat([df,temp],axis = 1)
            ol = pd.concat([ol,ol_temp],axis = 1)
            # shift the timeframe
            overlap_start = iterate
            end -= (delta-overlap)
            iterate -= (delta-overlap)
            time.sleep(2) #if 0, or removed google will think you are a bot so make sure it's greater than 1
        # averages for periods that were overlapped 
        df = df.mean(axis=1)
        ol = ol.max(axis=1)
        # merge the two dataframe (trend data and overlap flag)
        df = pd.concat([df,ol], axis=1)
        df.columns = [colname,'overlap'] #overlap value isn't useful (1.0 or NaN)
        df = df[start:intitial_end]
        # rescale to 100 (optional)
        df[colname] = (100*df[colname]/df[colname].max()).round(decimals=0)
        df = df.drop(['overlap'],axis=1)
        try:
            if os.stat(csvfile).st_size > 0:
                df.to_csv(csvfile, mode='a', header=False)
            else:
                #file should never be empty
                print("empty file")
        except OSError:
            df.to_csv(csvfile,index=True)

Each pair of csvs (i.e. initial climate change and ajdusted climate change csvs) are then concatenated into one big csv. Extra date columns are removed as they are redundant. (Optional)

In [ ]:
# this cell is optional
count = 0
for i in range(len(allcsv)):
    allData = []
    df = pd.read_csv(allcsv[i])
    df1 = pd.read_csv(allcsvadj[i])
    if count > 0:
        df = df.drop('date',axis=1)
        df1 = df1.drop('date',axis=1)
    allData.append(df)
    allData.append(df1)
    count +=1
    filename = 'aggregate' + allcsv[i] + '.csv'
    compare = pd.concat(allData, axis=1)
    compare.to_csv(filename,index=False)